Chapter 21
Creating and Updating Columns

In [1]:
# deletes variables
%reset -f

In [46]:
import pandas as pd

pd.set_option('display.max_rows', 10)             
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
    '2020-jetbrains-python-survey.csv'

jb = pd.read_csv(url)



C:\Users\wgrom\AppData\Local\Temp\ipykernel_9176\489044362.py:7: DtypeWarning: Columns (152) have mixed types. Specify dtype option on import or set low_memory=False.
  jb = pd.read_csv(url)


In [47]:

jb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54462 entries, 0 to 54461
Columns: 264 entries, is.python.main to country.live
dtypes: float64(1), object(263)
memory usage: 109.7+ MB


In [48]:
# create a series to look at names of columns 
cols = pd.Series(jb.columns)

In [49]:
# determine if a feature( column) can have multiple values(like database) and remove those.
import collections
counter = collections.defaultdict(list)

for col in sorted(jb.columns):
    period_count = col.count('.')
    if period_count >= 2:
        part_end = 2
    else:
        part_end = 1
    parts = col.split('.')[:part_end]
    temp = '.'.join(parts)
    counter['.'.join(parts)].append(col)




In [50]:
uniq_cols = []
for cols in counter.values():
    if len(cols) ==1:
        uniq_cols.extend(cols)

In [8]:
uniq_cols

['age',
 'are.you.datascientist',
 'company.size',
 'country.live',
 'employment.status',
 'first.learn.about.main.ide',
 'how.often.use.main.ide',
 'ide.main',
 'is.python.main',
 'job.team',
 'main.purposes',
 'missing.features.main.ide',
 'nps.main.ide',
 'python.years',
 'python2.version.most',
 'python3.version.most',
 'several.projects',
 'team.size',
 'use.python.most',
 'years.of.coding']

In [51]:
# the age column
# note when type age the autocomple want to use aggregate use the esc key to stop this
(jb[uniq_cols]
 .rename(columns=lambda c: c.replace('.','_'))
 .age
 .value_counts(dropna=False)
 )

NaN            29701
21–29           9710
30–39           7512
40–49           3010
18–20           2567
50–59           1374
60 or older      588
Name: age, dtype: int64

In [53]:
# can't convert to Int64 because of the bug
# https://github.com/pandas-dev/pandas/issues/33254
# but you can chain it like below
(jb[uniq_cols]
 .rename(columns=lambda c: c.replace('.','_'))
 .age
 .str.slice(0,2)  # can also use str[0:2]
 .astype(float)
 #.astype('Int64')   
 )

0        30.0
1        21.0
2        30.0
3         NaN
4        21.0
         ... 
54457    21.0
54458     NaN
54459    21.0
54460    30.0
54461    21.0
Name: age, Length: 54462, dtype: float64

In [54]:
(jb[uniq_cols]
 .rename(columns=lambda c: c.replace('.','_'))
 .age
 .str.slice(0,2)  # can also use str[0:2]
 .astype(float)
 .astype('Int64')   
 )

0          30
1          21
2          30
3        <NA>
4          21
         ... 
54457      21
54458    <NA>
54459      21
54460      30
54461      21
Name: age, Length: 54462, dtype: Int64

Chaining 
https://pandas.pydata.org/docs/user_guide/indexing.html#returning-a-view-versus-a-copy

In [12]:
import numpy as np

(jb[uniq_cols]
 .rename(columns=lambda c: c.replace('.','_'))
 .assign(age=lambda df_:df_.age.str.slice(0,2)
         .astype(float) .astype('Int64'),
         are_you_datascientist=lambda df_:df_.are_you_datascientist
         .replace({'Yes': True, 'No': False, np.nan: False})
        )
    .are_you_datascientist
 )

0        False
1         True
2        False
3        False
4        False
         ...  
54457    False
54458    False
54459    False
54460     True
54461    False
Name: are_you_datascientist, Length: 54462, dtype: object

In [13]:
(jb[uniq_cols]
 .rename(columns=lambda c: c.replace('.','_'))
 .assign(age=lambda df_:df_.age.str.slice(0,2)
         .astype(float) .astype('Int64'),
         are_you_datascientist=lambda df_:df_.are_you_datascientist
         .replace({'Yes': True, 'No': False, np.nan: False})
        )
    .company_size
    .value_counts(dropna=False)
 )

NaN                35037
51–500              4608
More than 5,000     3635
11–50               3507
2–10                2558
1,001–5,000         1934
Just me             1492
501–1,000           1165
Not sure             526
Name: company_size, dtype: int64

In [14]:


company_size=lambda df_:df_.company_size.replace({'Just me': 1,
    'Not sure': np.nan, 'More than 5,000': 5000, '2-10': 2, '11-50': 11, 
    '51-500': 51, '501-1,000': 501,
    '1,001-5000': 1001})


In [58]:
jb2 = (jb[uniq_cols]
 .rename(columns=lambda c: c.replace('.','_'))
 .assign(age=lambda df_:df_.age.str.slice(0,2)
         .astype(float) .astype('Int64'),
         are_you_datascientist=lambda df_:df_.are_you_datascientist
         .replace({'Yes': True, 'No': False, np.nan: False}),
         company_size=lambda df_:df_.company_size.replace({
            'Just me': 1, 'Not sure': np.nan, 
            'More than 5,000': 5000, '2–10': 2, '11–50': 11, 
            '51–500': 51, '501–1,000': 501,
            '1,001–5,000': 1001}).astype('Int64'),
            country_live=lambda df_:df_.country_live.astype('category'),
            employment_status=lambda df_:df_.employment_status
               .fillna('Other').astype('category'),
            is_python_main=lambda df_:df_.is_python_main.astype('category'),
            team_size=lambda df_:df_.team_size
               .str.split(r'-', n=1, expand=True)
               .iloc[:,0].replace('More than 40 people', 41)
               .where(df_.company_size!=1, 1).astype(float),
            years_of_coding=lambda df_:df_.years_of_coding
               .replace('Less than 1 year', .5).str.extract(r'(\d+)')
               .astype(float),
            python_years=lambda df_:df_.python_years
               .replace('Less than 1 year', .5).str.extract(r'(\d+)')
               .astype(float),
            python3_ver=lambda df_:df_.python3_version_most
               .str.replace('_', '.').str.extract(r'(\d\.\d)')
               .astype(float),
            use_python_most=lambda df_:df_.use_python_most
               .fillna('Unknow')

        )
   .drop(columns=['python2_version_most'])
    #.team_size
    #.value_counts(dropna=False)
 )

In [57]:
jb2

,age,are_you_datascientist,company_size,country_live,employment_status,first_learn_about_main_ide,how_often_use_main_ide,ide_main,is_python_main,job_team,main_purposes,missing_features_main_ide,nps_main_ide,python_years,python3_version_most,several_projects,team_size,use_python_most,years_of_coding,python3_ver
0,30,False,1,NaN,Partially employed by a company / organization,Conference / User Group,Weekly,PyCharm Community Edition,Yes,Work as an external consultant or trainer,For work,"No, it has all the features I need",3.0,NaN,Python 3_7,"Yes, I work on many different projects",1.0,Unknow,1.0,3.7
1,21,True,5000,India,Fully employed by a company / organization,School / University,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",8.0,NaN,Python 3_6,"Yes, I work on one main and several side projects",2.0,Software prototyping,3.0,3.6
2,30,False,5000,United States,Fully employed by a company / organization,Friend / Colleague,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,NaN,Python 3_6,"Yes, I work on one main and several side projects",NaN,DevOps / System administration / Writing autom...,3.0,3.6
3,<NA>,False,<NA>,NaN,Other,Friend / Colleague,Daily,PyCharm Professional Edition,Yes,NaN,Both for work and personal,Yes – Please list:,10.0,NaN,Python 3_8,"Yes, I work on many different projects",1.0,Web development,11.0,3.8
4,21,False,<NA>,Italy,Student,Search engines,Daily,VS Code,Yes,Work on your own project(s) independently,"For personal, educational or side projects","No, it has all the features I need",10.0,NaN,Python 3_8,"Yes, I work on one main and several side projects",1.0,Web development,NaN,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54457,21,False,2,Russian Federation,Fully employed by a company / organization,School / University,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,NaN,Python 3_6,"Yes, I work on many different projects",NaN,Data analysis,1.0,3.6
54458,<NA>,False,<NA>,NaN,Other,NaN,NaN,NaN,Yes,NaN,Both for work and personal,NaN,NaN,NaN,Python 3_7,NaN,1.0,Web development,1.0,3.7
54459,21,False,1,Russian Federation,Self-employed (a person earning income directl...,Friend / Colleague,Daily,PyCharm Professional Edition,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,NaN,Python 3_7,"Yes, I work on many different projects",1.0,Web development,6.0,3.7
54460,30,True,51,Spain,Fully employed by a company / organization,Search engines,Daily,Other,Yes,Work on your own project(s) independently,Both for work and personal,Yes – Please list:,3.0,NaN,Python 3_7,"Yes, I work on many different projects",NaN,Data analysis,3.0,3.7


In [45]:
(jb2
 .query('team_size.isna()')
 .employment_status
 .value_counts(dropna=False)
)

Freelancer (a person pursuing a profession without a long-term commitment to any one employer)    0
Fully employed by a company / organization                                                        0
Other                                                                                             0
Partially employed by a company / organization                                                    0
Retired                                                                                           0
Self-employed (a person earning income directly from one's own business, trade, or profession)    0
Student                                                                                           0
Working student                                                                                   0
Name: employment_status, dtype: int64

In [77]:
import catboost as cb
import numpy as np

import pandas as pd

import collections

In [78]:
def get_uniq_cols(jb):
    counter = collections.defaultdict(list)
    for col in sorted(jb.columns):
        period_count = col.count('.')
        if period_count >= 2:
            part_end = 2
        else:
            part_end = 1
        parts = col.split('.')[:part_end]
        counter['.'.join(parts)].append(col)
    uniq_cols = []
    for cols in counter.values():
        if len(cols) == 1:
            uniq_cols.extend(cols)
    return uniq_cols




In [79]:
def prep_for_ml(df):
    #remove pandas types  float, object and category
    return (df
            .assign(**{col:df[col].astype(float) for col in df.select_dtypes('number')},
                    **{col:df[col].astype(str) for col in df.select_dtypes(['object', 'category'])})
    )


   

In [80]:
def predict_col(df, col):
    #predict the missing values for team_size
    df = prep_for_ml(df)
    missing = df.query(f'~{col}.isna()')
    cat_idx = []
    for i,typ in enumerate(df.drop(columns=[col]).dtypes):
                if str(typ) == 'object':
                    cat_idx.append(i)
    X = (missing
         .drop(columns=[col])
         .values
         )
    y = missing[col]
    model = cb.CatBoostRegressor(iterations=20, cat_features=cat_idx)
    model.fit(X, y, cat_features=cat_idx)
    pred = model.predict(df.drop(columns=[col]))
    return df[col].where(~df[col].isna(), pred)

In [84]:
def tweak_jb(jb):
   uniq_cols = get_uniq_cols(jb)

   return  (jb[uniq_cols]
   .rename(columns=lambda c: c.replace('.','_'))
   .assign(age=lambda df_:df_.age.str.slice(0,2)
            .astype(float) .astype('Int64'),
            are_you_datascientist=lambda df_:df_.are_you_datascientist
            .replace({'Yes': True, 'No': False, np.nan: False}),
            company_size=lambda df_:df_.company_size.replace({
               'Just me': 1, 'Not sure': np.nan, 
               'More than 5,000': 5000, '2–10': 2, '11–50': 11, 
               '51–500': 51, '501–1,000': 501,
               '1,001–5,000': 1001}).astype('Int64'),
               country_live=lambda df_:df_.country_live.astype('category'),
               employment_status=lambda df_:df_.employment_status
                  .fillna('Other').astype('category'),
               is_python_main=lambda df_:df_.is_python_main.astype('category'),
               team_size=lambda df_:df_.team_size
                  .str.split(r'-', n=1, expand=True)
                  .iloc[:,0].replace('More than 40 people', 41)
                  .where(df_.company_size!=1, 1).astype(float),
               years_of_coding=lambda df_:df_.years_of_coding
                  .replace('Less than 1 year', .5).str.extract(r'(\d+)')
                  .astype(float),
               python_years=lambda df_:df_.python_years
                  .replace('Less than 1 year', .5).str.extract(r'(\d+)')
                  .astype(float),
               python3_ver=lambda df_:df_.python3_version_most
                  .str.replace('_', '.').str.extract(r'(\d\.\d)')
                  .astype(float),
               use_python_most=lambda df_:df_.use_python_most
                  .fillna('Unknown')

         )
      .assign(team_size=lambda df_:predict_col(df_, 'team_size')
            .astype(int))
      .drop(columns=['python2_version_most'])
      .dropna()
)

In [85]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
    '2020-jetbrains-python-survey.csv'

jb = pd.read_csv(url)

jb2 = tweak_jb(jb)

C:\Users\wgrom\AppData\Local\Temp\ipykernel_9176\2340151257.py:4: DtypeWarning: Columns (152) have mixed types. Specify dtype option on import or set low_memory=False.
  jb = pd.read_csv(url)


Learning rate set to 0.5
0:	learn: 2.9695218	total: 38.1ms	remaining: 723ms
1:	learn: 2.8766539	total: 80.1ms	remaining: 721ms
2:	learn: 2.8387189	total: 117ms	remaining: 661ms
3:	learn: 2.8028751	total: 154ms	remaining: 617ms
4:	learn: 2.7899957	total: 190ms	remaining: 569ms
5:	learn: 2.7749439	total: 226ms	remaining: 526ms
6:	learn: 2.7719128	total: 268ms	remaining: 497ms
7:	learn: 2.7649792	total: 302ms	remaining: 453ms
8:	learn: 2.7649588	total: 330ms	remaining: 404ms
9:	learn: 2.7630617	total: 366ms	remaining: 366ms
10:	learn: 2.7625779	total: 400ms	remaining: 327ms
11:	learn: 2.7515902	total: 437ms	remaining: 292ms
12:	learn: 2.7513459	total: 475ms	remaining: 256ms
13:	learn: 2.7445634	total: 511ms	remaining: 219ms
14:	learn: 2.7443257	total: 543ms	remaining: 181ms
15:	learn: 2.7423142	total: 577ms	remaining: 144ms
16:	learn: 2.7419143	total: 613ms	remaining: 108ms
17:	learn: 2.7399387	total: 652ms	remaining: 72.4ms
18:	learn: 2.7384296	total: 704ms	remaining: 37.1ms
19:	learn: 2

In [87]:
jb2

,age,are_you_datascientist,company_size,country_live,employment_status,first_learn_about_main_ide,how_often_use_main_ide,ide_main,is_python_main,job_team,main_purposes,missing_features_main_ide,nps_main_ide,python_years,python3_version_most,several_projects,team_size,use_python_most,years_of_coding,python3_ver
1,21,True,5000,India,Fully employed by a company / organization,School / University,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",8.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",2,Software prototyping,3.0,3.6
2,30,False,5000,United States,Fully employed by a company / organization,Friend / Colleague,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",4,DevOps / System administration / Writing autom...,3.0,3.6
10,21,False,51,Other country,Fully employed by a company / organization,School / University,Daily,IntelliJ IDEA,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,1.0,Python 3_8,"Yes, I work on one main and several side projects",2,Web development,1.0,3.8
11,21,True,51,United States,Fully employed by a company / organization,Online learning platform / Online course,Daily,PyCharm Community Edition,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",9.0,3.0,Python 3_9,"Yes, I work on many different projects",2,Data analysis,3.0,3.9
13,30,True,5000,Belgium,Fully employed by a company / organization,Social network,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,6.0,Python 3_7,"Yes, I work on many different projects",2,Data analysis,3.0,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54456,30,False,1001,Turkey,Fully employed by a company / organization,Friend / Colleague,Daily,PyCharm Community Edition,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",9.0,1.0,Python 3_6,"Yes, I work on many different projects",5,Machine learning,6.0,3.6
54457,21,False,2,Russian Federation,Fully employed by a company / organization,School / University,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,6.0,Python 3_6,"Yes, I work on many different projects",2,Data analysis,1.0,3.6
54459,21,False,1,Russian Federation,Self-employed (a person earning income directl...,Friend / Colleague,Daily,PyCharm Professional Edition,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,3.0,Python 3_7,"Yes, I work on many different projects",1,Web development,6.0,3.7
54460,30,True,51,Spain,Fully employed by a company / organization,Search engines,Daily,Other,Yes,Work on your own project(s) independently,Both for work and personal,Yes – Please list:,3.0,6.0,Python 3_7,"Yes, I work on many different projects",4,Data analysis,3.0,3.7
